In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from google.colab import drive
%run "/content/drive/MyDrive/Colab Notebooks/functions.ipynb"

drive.mount('/content/drive')
df_airlines = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/airlines.csv')
df_airports = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/airports.csv')
df_flights = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/flights.csv',
                         dtype={'ORIGIN_AIRPORT': str,
                                'DESTINATION_AIRPORT': str})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df_flights.copy()

In [ ]:
columns_hour_to_minutes = ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL']

df_deleted = df[df['CANCELLED'].notna() | df['DIVERTED'].notna()]
df = df[~((df['CANCELLED'] == 1
                           ) | (df['DIVERTED'] == 1))]

for column in columns_hour_to_minutes:
  convert_hour_to_minutes(df, column)

for column in columns_hour_to_minutes:
  encode_cyclic_feature(df, column, 1440)

encode_cyclic_feature(df, 'DAY', 31)
encode_cyclic_feature(df, 'DAY_OF_WEEK', 7)
encode_cyclic_feature(df, 'MONTH', 12)

label_encoder = LabelEncoder()

df_airports['ID'] = label_encoder.fit_transform(df_airports['IATA_CODE'])
df_airlines['ID'] = label_encoder.fit_transform(df_airlines['IATA_CODE'])
df['ID_TAIL_NUMBER'] = label_encoder.fit_transform(df['TAIL_NUMBER'])

airpot_map = df_airports.set_index('IATA_CODE')['ID'].to_dict()
airlines_map = df_airlines.set_index('IATA_CODE')['ID'].to_dict()

df['ID_ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].map(airpot_map)
df['ID_DESTINATION_AIRPORT'] = df['DESTINATION_AIRPORT'].map(airpot_map)
df['ID_AIRLINE'] = df['AIRLINE'].map(airlines_map)
df['ID_ORIGIN_AIRPORT'].fillna(df['ORIGIN_AIRPORT'], inplace=True)
df['ID_DESTINATION_AIRPORT'].fillna(df['DESTINATION_AIRPORT'], inplace=True)
df['ID_ARRIVAL_DELAY'] = df['ARRIVAL_DELAY'].apply(categorize_value)
df['ID_ORIGIN_AIRPORT'] = df['ID_ORIGIN_AIRPORT'].astype(int)
df['ID_DESTINATION_AIRPORT'] = df['ID_DESTINATION_AIRPORT'].astype(int)

df = df.astype({
                'DAY_sin': 'float32',
                'DAY_cos': 'float32',
                'DAY_OF_WEEK_sin': 'float32',
                'DAY_OF_WEEK_cos': 'float32',
                'MONTH_sin': 'float32',
                'MONTH_cos': 'float32',
                'ID_AIRLINE': 'int16',
                'FLIGHT_NUMBER': 'int16',
                'ID_TAIL_NUMBER': 'int16',
                'SCHEDULED_DEPARTURE_sin': 'float32',
                'SCHEDULED_DEPARTURE_cos': 'float32',
                'ID_ORIGIN_AIRPORT': 'int16',
                'ID_DESTINATION_AIRPORT': 'int16',
                'SCHEDULED_TIME': 'float32',
                'SCHEDULED_ARRIVAL_sin': 'float32',
                'SCHEDULED_ARRIVAL_cos': 'float32',
                'DISTANCE': 'int16'
})


X = df[['DAY_sin', 'DAY_cos', 'DAY_OF_WEEK_sin', 'DAY_OF_WEEK_cos', 'MONTH_sin',
        'MONTH_cos', 'ID_AIRLINE', 'FLIGHT_NUMBER', 'ID_TAIL_NUMBER',
        'SCHEDULED_DEPARTURE_sin', 'SCHEDULED_DEPARTURE_cos',
        'ID_ORIGIN_AIRPORT', 'ID_DESTINATION_AIRPORT', 'SCHEDULED_TIME',
        'SCHEDULED_ARRIVAL_sin', 'SCHEDULED_ARRIVAL_cos', 'DISTANCE']]
y = df['ID_ARRIVAL_DELAY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.2,
                                                  random_state=42)

In [ ]:
del df_flights
del X
del y
del df
del df_airports
del df_airlines
del airlines_map
del airpot_map